# Vasculature simulation use-case

### Connect to back-end

In [6]:
from brayns import Client
from circuitexplorer import CircuitExplorer
from bioexplorer import BioExplorer, TransferFunction, MovieMaker

url = 'localhost:5000'

brayns = Client(url)
ce = CircuitExplorer(brayns)
be = BioExplorer(url)
mm = MovieMaker(be)

### Load vasculature

In [7]:
data_folder = '/gpfs/bbp.cscs.ch/project/proj129/medias/ngv/vasculature/'

In [8]:
vasculature_file = data_folder + 'graph_with_endfeet.h5'
vasculature_model = ce.load_vasculature(
    name='Vasculature', path=vasculature_file,
    use_sdf=False, geometry_quality=ce.GEOMETRY_QUALITY_HIGH,
    color_scheme=ce.MORPHOLOGY_COLOR_SCHEME_NONE
)
vasculature_model_id = vasculature_model['id']

In [9]:
ce.set_vasculature_default_materials(
    model_id=vasculature_model_id, palette_name='Set1',
    shading_mode=ce.SHADING_MODE_NONE
)

### Sections

In [ ]:
ce.set_vasculature_section_color(
    model_id=vasculature_model_id,
    section_id=10000,
    color=[1,0,0]
)

In [ ]:
for i in range(10000):
    ce.set_vasculature_section_color(
        model_id=vasculature_model_id,
        section_id=i,
        color=[1,1,1]
    )

### Segments

In [ ]:
ce.set_vasculature_segment_color(
    model_id=vasculature_model_id,
    segment_id=0,
    color=[1,0,0]
)

### Set camera default position and rendering parameters

In [4]:
status = brayns.set_camera(
    orientation = [-0.188, 0.28, 0.036, 0.940],
    position = [1577, 2183, 2511], target = [333, 1279, 646])

In [5]:
status = brayns.set_renderer(
    current='circuit_explorer_advanced',
    subsampling=4, max_accum_frames=256)
params = brayns.CircuitExplorerAdvancedRendererParams()
params.epsilon_factor = 1000.0
params.shadows = 1.0
params.soft_shadows = 1.0
status = brayns.set_renderer_params(params)

## Simulation report

### Simulation transfer function

In [ ]:
tf = TransferFunction(
    bioexplorer=BioExplorer(url), name='Reds',
    model_id=vasculature_model['id'], size=16, alpha=1.0
)

### Set materials to display simulation data

In [ ]:
ce.set_vasculature_default_materials(
    model_id=vasculature_model_id, palette_name='Reds',
    shading_mode=ce.SHADING_MODE_NONE,
    map_simulation_data=True)

### Attach simulation report to the vasculature

In [ ]:
''' Report file names and value ranges '''
reports = ['report_flows', 'report_pressures', 'report_radii']
reports_ranges = [[-0.5, 1.5], [0, 0.0015], [0, 45]]

''' Attach report to vasculature '''
report_id = 0

report_filename = data_folder + reports[report_id] +'.h5'
ce.attach_vasculature_report(
    model_id=vasculature_model_id,
    path=report_filename, debug=False)
tf.set_range(reports_ranges[report_id])

### Widget example

In [ ]:
from ipywidgets import Select, HBox

def update_report(value):
    ce.attach_vasculature_report(
        model_id=vasculature_model_id,
        path=data_folder + reports[value.new] + '.h5')
    tf.set_palette(ce.COLOR_MAPS[colormap_selector.index])
    tf.set_range(reports_ranges[report_selector.index])
    brayns.set_renderer()
    
def update_colormap(value):
    tf.set_palette(ce.COLOR_MAPS[colormap_selector.index])
    tf.set_range(reports_ranges[report_selector.index])
    brayns.set_renderer()

report_selector = Select(options=reports)
report_selector.observe(update_report, 'index')
colormap_selector = Select(options=ce.COLOR_MAPS)
colormap_selector.observe(update_colormap, 'index')
hbox = HBox([report_selector, colormap_selector])
display(hbox)

## Snapshots

### Single snapshot

In [ ]:
status = brayns.set_camera(
    orientation = [-0.188, 0.280, 0.035, 0.940],
    position = [1875, 2400, 2958], target = [333, 1279, 646])

In [ ]:
''' Create image and save it to disk '''
mm.create_snapshot(
    size=[2160, 3840], samples_per_pixel=64,
    base_name='vasculature_4k_v4', path='/tmp')

### Zoom

In [ ]:
status = brayns.set_camera(
    orientation = [-0.188, 0.280, 0.035, 0.940],
    position=[879, 2075, 1188], target=[366, 1702, 418])

In [ ]:
''' Create image and save it to disk '''
mm.create_snapshot(
    size=[3840, 2160], samples_per_pixel=64,
    base_name='vasculature_4k_v5', path='/tmp')

### Simple movie

In [ ]:
for frame in range(720):
    ''' Apply geometry modifications to the vasculature '''
    ce.apply_vasculature_geometry_report(
        path=data_folder + 'report_radii.h5',
        model_id=vasculature_model_id,
        frame=frame, amplitude=0.75, debug=True
    )
    ''' Set animation frame (for the mapping of the simulation report) '''
    brayns.set_animation_parameters(current=frame)

    ''' Create image and save it to disk '''
    mm.create_snapshot(
        size=[1080, 1080], samples_per_pixel=32,
        base_name='%05d' % frame, path='/tmp')